In [57]:
import psycopg2
import psycopg2.extras as extras
import pandas as pd
import json
from datetime import datetime
from google.cloud import bigquery

#pip uninstall psycopg
#pip install psycopg2

In [58]:
#company_id_query=(2,16,17,18,19) # AIS,BBL,INET- (HPE Product)
company_id_query=(2,) # ais 

In [59]:
table_id = "pongthorn.SMartAppData.incident_info"
client = bigquery.Client()

In [60]:
#start_date_query='2000-01-01' 
start_date_query='2022-01-01'  # last exported data

In [61]:
dt_imported=datetime.now()
str_imported=dt_imported.strftime('%Y-%m-%d %H:%M:%S')
print(f"Imported DateTime: {str_imported}" )


Imported DateTime: 2022-09-15 22:58:30


In [62]:
sql_lastImport=f"SELECT max(imported_at) as last_imported from `pongthorn.SMartAppData.incident_info` where open_datetime>='{start_date_query}' "
job_lastImported=client.query(sql_lastImport)
str_lastImported=None
for row in job_lastImported:    
    if row.last_imported is not None: 
        str_lastImported=row.last_imported.strftime('%Y-%m-%d %H:%M:%S')
print(f"Last Imported DateTime: {str_lastImported}" )

Last Imported DateTime: None


In [63]:
if str_lastImported is not None:
  start_date_query=str_lastImported

print(f"Start Import on update_at of last imported date : {start_date_query}" )

Start Import on update_at of last imported date : 2022-01-01


In [64]:
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
         database='SMartDB', user='postgres',
      password='P@ssw0rd', host='localhost', 
     )
  return conn

 except Exception as error:
  print(error)      
  raise error
def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

In [65]:
#incident.id,  order by id

sql_incident="""
select  
incident.id ,
company.company_name as company,incident.incident_no as incident_no, 

product_type.productype_name,brand.brand_name,model.model_name,
inventory.serial_number,datacenter.datacenter_name,branch.branch_name,

incident.incident_severity_id as severity_id,severity.severity_name ,
incident.incident_status_id as status_id,
status.incident_status_name as status_name,

incident.service_type_id,service.service_type_name,
incident.incident_type_id,xtype.incident_type_name,
failure_type,

TO_CHAR(incident.incident_datetime,'YYYY-MM-DD HH24:MI:SS') as open_datetime,
TO_CHAR(incident.incident_close_datetime,'YYYY-MM-DD HH24:MI:SS') as close_datetime,

TO_CHAR(incident.incident_problem_start,'YYYY-MM-DD HH24:MI:SS') as response_datetime,
TO_CHAR(incident.incident_problem_end,'YYYY-MM-DD HH24:MI:SS') as resolved_datetime,
TO_CHAR(incident.updated_at,'YYYY-MM-DD HH24:MI:SS') as updated_at ,


TO_CHAR(inventory.install_date,'YYYY-MM-DD HH24:MI:SS') as install_date,
TO_CHAR(inventory.eos_date,'YYYY-MM-DD HH24:MI:SS') as eos_date,
TO_CHAR(inventory.customer_warranty_start,'YYYY-MM-DD HH24:MI:SS') as customer_warranty_start,
TO_CHAR(inventory.customer_warranty_end,'YYYY-MM-DD HH24:MI:SS') as customer_warranty_end


from app_incident as incident
inner join  app_incident_status as status on incident.incident_status_id = status.id
inner join  app_incident_severity as severity on  incident.incident_severity_id = severity.id
inner join  app_service_type as service on incident.service_type_id= service.id
inner join  app_incident_type as  xtype on incident.incident_type_id = xtype.id


inner join app_inventory as inventory on incident.inventory_id = inventory.id
inner join app_brand as brand on inventory.brand_id = brand.id
inner join app_model as model on inventory.model_id = model.id
inner join app_product_type as product_type on inventory.product_type_id = product_type.id
inner join app_datacenter as datacenter on inventory.datacenter_id = datacenter.id
inner join app_branch as branch on inventory.branch_id = branch.id

inner join app_project as project on inventory.project_id = project.id
inner join app_company as company on project.company_id = company.id

where  incident.updated_at > %(start_date_param)s  

and company.id in %(company_id_param)s
and incident.incident_status_id=4

order by  incident.updated_at desc



"""

#and  incident.incident_datetime<=%(end_date_param)s

In [66]:

sql_detail="""
select  detail.id, detail.incident_master_id as incident_id ,
TO_CHAR(task_start,'YYYY-MM-DD HH24:MI:SS') as task_start,
TO_CHAR(task_end,'YYYY-MM-DD HH24:MI:SS') as task_end,
team.service_team_name,
engineer.employee_name as engineer_name,
TO_CHAR(detail.updated_at,'YYYY-MM-DD HH24:MI:SS') as updated_at 

from app_incident_detail detail
inner join  app_serviceteam team on detail.service_team_id=team.id
inner  join  app_employee engineer on detail.employee_id=engineer.id
 where detail.incident_master_id = %(incident_id_param)s 
 """

#,detail."reference_product_caseNo" as case_id 

In [67]:
print("Create all issues dataframe")

# dict_params={"company_id_param":company_id_query,"start_date_param":start_date_query,"end_date_param":end_date_query}
dict_params={"company_id_param":company_id_query,"start_date_param":start_date_query}

df_all=list_data(sql_incident,dict_params,get_postgres_conn())
df_all['imported_at']=dt_imported
df_all['imported_at']=df_all['imported_at'].dt.strftime('%Y-%m-%d %H:%M:%S')

dateNone_cols=['close_datetime','resolved_datetime']
for index,row in df_all.iterrows():
  for dateNone in dateNone_cols:
    if  row[dateNone] is None:
        print(row['incident_no']+ " is updated on None in "+dateNone)
        df_all.loc[index, dateNone] =row['updated_at']

Create all issues dataframe


In [68]:
#df_all=df_all.head(10)
print(df_all.info())
df_all.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       302 non-null    int64 
 1   company                  302 non-null    object
 2   incident_no              302 non-null    object
 3   productype_name          302 non-null    object
 4   brand_name               302 non-null    object
 5   model_name               302 non-null    object
 6   serial_number            302 non-null    object
 7   datacenter_name          302 non-null    object
 8   branch_name              302 non-null    object
 9   severity_id              302 non-null    int64 
 10  severity_name            302 non-null    object
 11  status_id                302 non-null    int64 
 12  status_name              302 non-null    object
 13  service_type_id          302 non-null    int64 
 14  service_type_name        302 non-null    o

,id,company,incident_no,productype_name,brand_name,model_name,serial_number,datacenter_name,branch_name,severity_id,...,open_datetime,close_datetime,response_datetime,resolved_datetime,updated_at,install_date,eos_date,customer_warranty_start,customer_warranty_end,imported_at
0,1438,AIS,SR-ES-22-1438,Server,HPE,ProLiant DL560 Gen10,SGH035S316,SILA1,AIS,2,...,2022-09-14 10:40:00,2022-09-15 17:03:00,2022-09-14 10:40:00,2022-09-14 18:05:00,2022-09-15 15:28:39,None,None,2021-05-17 00:00:00,2026-05-27 00:00:00,2022-09-15 22:58:30
1,1386,AIS,SR-ES-22-1386,Software,TrendMicro,Apex One Endpoint,ApexOne10001,SILA1,AIS,3,...,2022-09-02 16:24:00,2022-09-15 12:34:00,2022-09-02 16:32:00,2022-09-15 12:34:00,2022-09-15 12:39:39,None,None,2022-05-01 00:00:00,2023-04-30 00:00:00,2022-09-15 22:58:30
2,1246,AIS,SR-ES-22-1246,Storage,NetApp,FAS8020,451436000071,TLS1,AIS,3,...,2022-08-05 18:06:00,2022-09-14 13:01:00,2022-08-05 18:06:00,2022-09-14 13:01:00,2022-09-15 12:06:53,2014-09-01 00:00:00,2023-01-31 00:00:00,2022-05-01 00:00:00,2023-04-30 00:00:00,2022-09-15 22:58:30
3,1440,AIS,SR-ES-22-1440,Software,VMWare,vRealize Automation Advanced Per OSI,vRA1300,SILA1,AIS,3,...,2022-09-14 14:58:00,2022-09-14 16:41:00,2022-09-14 15:03:00,2022-09-14 16:41:00,2022-09-14 17:21:23,None,2018-08-01 00:00:00,2022-07-30 00:00:00,2023-07-29 00:00:00,2022-09-15 22:58:30
4,1434,AIS,SR-ES-22-1434,Software,VMWare,vRealize Automation Advanced Per OSI,vRA1300,SILA1,AIS,3,...,2022-09-13 11:29:00,2022-09-13 13:59:00,2022-09-13 13:03:00,2022-09-13 13:59:00,2022-09-13 14:51:46,None,2018-08-01 00:00:00,2022-07-30 00:00:00,2023-07-29 00:00:00,2022-09-15 22:58:30
5,1380,AIS,SR-ES-22-1380,Storage,NetApp,FAS8020,451608000206,TLS1,AIS,3,...,2022-09-01 14:48:00,2022-09-12 09:32:00,2022-09-01 14:48:00,2022-09-12 09:32:00,2022-09-12 11:18:49,2016-02-01 00:00:00,2023-01-31 00:00:00,2022-05-01 00:00:00,2023-04-30 00:00:00,2022-09-15 22:58:30
6,1403,AIS,SR-ES-22-1403,Software,VMWare,vRealize Automation Advanced Per OSI,vRA1300,SILA1,AIS,3,...,2022-09-07 09:26:00,2022-09-07 10:17:00,2022-09-07 09:32:00,2022-09-07 10:17:00,2022-09-08 08:33:18,None,2018-08-01 00:00:00,2022-07-30 00:00:00,2023-07-29 00:00:00,2022-09-15 22:58:30
7,1404,AIS,SR-ES-22-1404,Software,VMWare,vRealize Automation Advanced Per OSI,vRA1300,SILA1,AIS,3,...,2022-09-07 13:38:00,2022-09-07 16:49:00,2022-09-07 14:19:00,2022-09-07 16:49:00,2022-09-07 16:52:05,None,2018-08-01 00:00:00,2022-07-30 00:00:00,2023-07-29 00:00:00,2022-09-15 22:58:30
8,1346,AIS,SR-ES-22-1346,Storage,NetApp,FAS8020,451525000152,SILA1,AIS,3,...,2022-08-25 12:43:00,2022-09-07 09:27:00,2022-08-25 12:43:00,2022-09-07 01:24:00,2022-09-07 09:35:09,None,2023-01-31 00:00:00,2022-05-01 00:00:00,2023-04-30 00:00:00,2022-09-15 22:58:30
9,1360,AIS,SR-ES-22-1360,Software,VMWare,vRealize Automation Advanced Per OSI,vRA1300,SILA1,AIS,3,...,2022-08-29 11:41:00,2022-09-06 21:10:00,2022-08-29 12:41:00,2022-09-06 21:10:00,2022-09-06 21:13:11,None,2018-08-01 00:00:00,2022-07-30 00:00:00,2023-07-29 00:00:00,2022-09-15 22:58:30


In [69]:
json_incident = json.loads(df_all.to_json(orient = 'records'))
for incident in json_incident:
    
    id = incident['id']
    incident.pop('id')
    
    incident_update_at= incident['imported_at']
    
    df_detail = list_data(sql_detail,{"incident_id_param": id},get_postgres_conn())
    
    if df_detail.empty==False:
    
      for index,row in df_detail.iterrows():
        if  row['task_end'] is None:
           print(f"DetailId-{row['id']} of IncidentID-{row['incident_id']}  is updated on None in task_end") 
           df_detail.loc[index, 'task_end'] =  incident_update_at #row['updated_at']
           
      df_detail=df_detail.drop(columns=['incident_id','id'])  
      json_detail = json.loads(df_detail.to_json(orient = 'records'))
      # print(json_detail)
      incident['incident_detail']=json_detail
        
    else:
     print(f'no detail in incident {id}')
     #incident['incident_detail']=None

DetailId-1444 of IncidentID-1004  is updated on None in task_end
DetailId-1470 of IncidentID-680  is updated on None in task_end
DetailId-1736 of IncidentID-1144  is updated on None in task_end
no detail in incident 1006
DetailId-1386 of IncidentID-978  is updated on None in task_end
DetailId-1350 of IncidentID-957  is updated on None in task_end
DetailId-1296 of IncidentID-903  is updated on None in task_end
DetailId-515 of IncidentID-479  is updated on None in task_end
DetailId-42 of IncidentID-34  is updated on None in task_end
DetailId-304 of IncidentID-34  is updated on None in task_end
DetailId-316 of IncidentID-303  is updated on None in task_end
DetailId-315 of IncidentID-303  is updated on None in task_end


In [70]:
# with open("incident.json", "w") as outfile:
#     json.dump(json_incident, outfile)

no_incident=len(json_incident)
print(f"No incident: {no_incident}")
if len(json_incident)>0:
    print(json_incident[0])

No incident: 302
{'company': 'AIS', 'incident_no': 'SR-ES-22-1438', 'productype_name': 'Server', 'brand_name': 'HPE', 'model_name': 'ProLiant DL560 Gen10', 'serial_number': 'SGH035S316', 'datacenter_name': 'SILA1', 'branch_name': 'AIS', 'severity_id': 2, 'severity_name': 'Major', 'status_id': 4, 'status_name': 'Closed', 'service_type_id': 1, 'service_type_name': 'Incident', 'incident_type_id': 20, 'incident_type_name': 'Other Failure', 'failure_type': 'Mainboard', 'open_datetime': '2022-09-14 10:40:00', 'close_datetime': '2022-09-15 17:03:00', 'response_datetime': '2022-09-14 10:40:00', 'resolved_datetime': '2022-09-14 18:05:00', 'updated_at': '2022-09-15 15:28:39', 'install_date': None, 'eos_date': None, 'customer_warranty_start': '2021-05-17 00:00:00', 'customer_warranty_end': '2026-05-27 00:00:00', 'imported_at': '2022-09-15 22:58:30', 'incident_detail': [{'task_start': '2022-09-15 15:03:00', 'task_end': '2022-09-15 15:03:00', 'service_team_name': 'Service Delivery Manager (SM)', 'e

In [71]:

job_config = bigquery.LoadJobConfig(
      schema = [              
    bigquery.SchemaField("company", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("incident_no", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("productype_name", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("brand_name", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("model_name", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("serial_number", "STRING", mode="REQUIRED"), 
    bigquery.SchemaField("datacenter_name", "STRING", mode="REQUIRED"), 
    bigquery.SchemaField("branch_name", "STRING", mode="REQUIRED"),       
    bigquery.SchemaField("severity_id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("severity_name", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("status_id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("status_name", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("service_type_id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("service_type_name", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("incident_type_id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("incident_type_name", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("failure_type", "STRING", mode="NULLABLE"),          
    bigquery.SchemaField("open_datetime", "DATETIME", mode="REQUIRED"),
    bigquery.SchemaField("close_datetime", "DATETIME", mode="REQUIRED"),
    bigquery.SchemaField("response_datetime", "DATETIME", mode="REQUIRED"),
    bigquery.SchemaField("resolved_datetime", "DATETIME", mode="REQUIRED"),
    bigquery.SchemaField("updated_at", "DATETIME", mode="REQUIRED"),
    bigquery.SchemaField("install_date", "DATETIME", mode="NULLABLE"),
    bigquery.SchemaField("eos_date", "DATETIME", mode="NULLABLE"),
    bigquery.SchemaField("customer_warranty_start", "DATETIME", mode="REQUIRED"),
    bigquery.SchemaField("customer_warranty_end", "DATETIME", mode="REQUIRED"),
    bigquery.SchemaField("imported_at", "DATETIME", mode="REQUIRED"),      
    bigquery.SchemaField(
    "incident_detail",
    "RECORD",
    mode="REPEATED",
    fields=[
        bigquery.SchemaField("task_start", "DATETIME", mode="REQUIRED"),
        bigquery.SchemaField("task_end", "DATETIME", mode="REQUIRED"),
        bigquery.SchemaField("service_team_name", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("engineer_name", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("updated_at", "DATETIME", mode="REQUIRED"),
    ],),    
    ]
   
)
# for meta in job_config.schema:
#  print(meta)

In [72]:
job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
#job_config.schema_update_options = [bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION]
job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND  
#job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

In [73]:

job = client.load_table_from_json(json_incident,table_id, job_config = job_config)
if job.errors is not None:
    print(job.error_result)
    print(job.errors)
else:
    print(f"import to bigquery successfully  {no_incident} records")

import to bigquery successfully  302 records
